# Домашнее задание 1. 

-  "Оптимальность" наивного Байесовского классификатора
- Оценка "важности" признаков (feature importance)

Для тех кто не знаком с библиотекой pandas я рекомендую вот эти два практических туториала:

-  https://nbviewer.jupyter.org/github/justmarkham/pandas-videos/blob/master/pandas.ipynb (1-3 часа на прохождение, 30 вопросов с видео разборами)
- https://github.com/guipsamora/pandas_exercises/ 11 топиков покрывающих весь основной функционал библиотеки


## 1. Наивный Байес

Ваша задача сравнить вероятности получаемые наивным байесовским классификатором с вероятностями получаемыми если не делать предположения о независимости признаков. Мы будем делать это для 2 признаков рассмотренных на семинаре: Пол и Класс билета (пассажира).

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('../data/titanic_data.csv')

In [ ]:
X = pd.DataFrame()

X['Sex'] = df['Sex'].map({
    'Male':0,
    'Female':1
})
X['PassengerClass'] = df['Passenger Class'].map({
    'First': 1,
    'Second': 2,
    'Third': 3
})

X['Survived'] = df['Survived'].map({
    'Yes': 1,
    'No': 0
})

y = X['Survived']
X = X.drop(['Survived'], axis=1)

## 1.1 Разделим данные на тренировочную и тестовую части 70-30

сколько наблюдений оказалось в тренировочной и тестовой выборках?

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.70, random_state=5)

## 1.2 Реализуйте функцию `get_probability_survived_bayes`

Которая считает честные вероятности выживания при условии заданного класса билета и пола:

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{Passenger Class}, \text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}|\text{Passenger Class}) p(\text{Passenger Class})}
$$

## Помните что оценка вероятностей всегда должна происходить на тренировчной выборке.

In [ ]:
from typing import Tuple

In [ ]:
# Hint 1. Используйте pandas.groupby
# Hint 2. Вы возвращаете tuple p_not_survived, p_survived, в котором
# вероятность выжить находится на второй позиции, чтобы когда вы берете аргмакс получать метки классов 0/1 (не выжил/выжил)

def get_probability_survived_bayes(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns true probabilities tuple(p_not_survived, p_survived)'''
    ...
    


## 1.3 Реализуйте функцию `get_probability_survived_bayes_naive`

Которая оценивает вероятность выжить в предположении о независимости признаков

$$
P(\text{y}|\text{Sex}, \text{Passenger Class}) = \frac{p(\text{Sex}|\text{y}) p(\text{Passenger Class}|\text{ y}) p(\text{y})}{p(\text{Sex}) p(\text{Passenger Class})}
$$

In [ ]:
def get_probability_survived_bayes_naive(pclass: int, sex: int) -> Tuple[float, float]:
    '''Given `Passenger Class` and `Sex` returns tuple(p_not_survived, p_survived), 
    under naive assumption of features independence'''
    ...
    
# Hint 1. Поскольку вы делаете предположении о независимости признаков, то p(y=1|...) + p(y=0| ...) некорректно отнормированы
# и не будут суммироваться в 1. Не забудьте выполнить необходимую нормировку.

## 1.4 Используйте ваши функции для того чтобы сделать предсказание на тесте


Для всех 6 уникальных комбинаций значений признаков сравните вероятности принадлежности классу 1
 - насколько они отличаются?
 - влияют ли эти отличия на предсказание класса объекта, при пороге $p = 0.5$?

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
y_pred_proba = []

for _, (sex, pclass, _) in X_test.iterrows():
    y_pred_proba.append(get_probability_survived_bayes_naive(pclass, sex))

## 1.5 Обучите имплементацию `MultinomialNB` из `sklearn`

Тот же вопрос что и 1.4, прокомментируйте отличия в оценке вероятности ваших имлементаций и имплементации
`MultinomialNB`. Влияют ли они на предсказание классов при стандартном пороге вероятности (p=0.5)? Заполните таблицу `probabilities`.

In [ ]:
probabilities = pd.DataFrame()
probabilities['Sex'] = [0,0,0,1,1,1]
probabilities['Passenger Class'] = [1,2,3,1,2,3]
probabilities['Bayes'] = [0]*6
probabilities['BayesNaive'] = [0]*6
probabilities['MultinomialNB'] = [0]*6

probabilities

## 2.1 Используя MultinomialNB обучите классификатор на бОльшем количестве признаков

1. Подготовьте признаки 
- 'Passenger Class',
- 'Sex',
- 'Age',
- 'No of Siblings or Spouses on Board',
- 'No of Parents or Children on Board',
- 'Port of Embarkation'

для обучения:
- закодируйте столбец `Port of Embarkation` по аналогии с `Passenger Class`
- заполните пропуски в столбце `Age` средним возрастом в датасете используйте `pd.DataFrame.fillna(...)`
- заполните пропуске в столбце `Port of Embarkation` самым частым портом посадки
- для разбиения на тренировчную и тестовую часть данных используйте `train_size=0.66` и `random_state=5`

2. обучите MultinomialNB модель на тренировочной части данных
2. Как изменилась точность вашей модели на тестовой части данных по сравнению с моделью обученной на 2 признаках?
3. Что означает признак `Life Boat`? Следуют ли на ваш взгляд включать его в список признаков для обучения модели? 
- Опишите сценарий (использования модели) в котором его следует включать в список признаков. 
- Опишите сценарий (использования модели) в котором его не следует включать в список признаков.
5. Какая ошибка содержится в предложенной последовательности действий в п. 2.1? 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.66, random_state=5)

## 2.2. Оценка важности признаков

Мы воспользуемся простым (и не очень хорошим^[1]) методом оценки важности признаков:

1. Воспользуйтесь моделью из п.2. Качество модели на тестовой части данных это ваш бейзлайн
2. Исключая по 1 признаку из `X` обучайте модель `MultinomialNB` и измеряйте ее точность на тестовой части данных
3. Сохраняйте величину падения или роста качества по сравнению с бейзлайном. Если точность упала значит признак "полезный", если выросло - "вредный". Абсолютная величина падения или роста в данном случае не так важна, важен только порядок признаков. Такая оценка называется качественной (в противовес количественной).
4. Прокомментируйте полученный результат, насколько он согласуется с вашей интуицией?
5. В чем на ваш взляд могут быть преимущества использования мЕньшего числа признаков при обучении моделей? (не обязательно в контексте данной задачи)


[1] а впрочем какие из них хороши?

jupyter notebook c вашим решением необходимо отправить на почту kurmukovai@gmail.com, с темой письма `[ml-ds-2024-ha1-Surname]`, например `[ml-ds-2023-Kurmukov]` до 12:00 МСК 08.02.2023. Задания отправленные до полуночи Вторника 06.02.2023 получают +1 балл в случае полного выполнения (не обязательно верного:))